In [2]:
sheets = pd.read_excel("FReD.xlsx", sheet_name=None)
print(sheets.keys())

dict_keys(['Data', 'DOI_dedup', 'FORRT R&R (editable)', 'Data (editable)', 'Copy of Data', 'Additional Studies to be added', 'Additional Studies to be added ', 'File Links', 'COS coding', 'Data validation', 'ES calculations', 'ORG - var renames', 'Contributors FReD', 'Roadmap', 'Check regularly for Replication', 'Key Variables'])


In [7]:
import pandas as pd
import requests
from pathlib import Path
import time, random
import re

def make_direct_download_url(url):
    """
    Convert a Google Drive 'view' or 'open?id=' link into a direct download link.
    """
    if not isinstance(url, str):
        return None

    # extract file ID
    match = re.search(r"/d/([a-zA-Z0-9_-]+)", url) or re.search(r"id=([a-zA-Z0-9_-]+)", url)
    if not match:
        return None

    file_id = match.group(1)
    return f"https://drive.usercontent.google.com/u/0/uc?id={file_id}&export=download"

# === Load data ===
df = pd.read_excel("FReD.xlsx", sheet_name="File Links")

output_dir = Path("/home/dan/FReD_existing_PDFs/")
output_dir.mkdir(exist_ok=True)

# === Download loop ===
for i, row in df.iterrows():
    url = row.get("URL")
    filename = str(row.get("lower")).strip()

    if not url or pd.isna(url):
        print(f"Skipping row {i}: missing URL")
        continue

    # Convert to direct download
    direct_url = make_direct_download_url(url)
    if not direct_url:
        print(f"Skipping row {i}: invalid Google Drive URL -> {url}")
        continue

    filepath = output_dir / filename
    if filepath.exists():
        print(f"Skipping (already exists): {filename}")
        continue

    try:
        print(f"Downloading {filename} ...")
        with requests.get(direct_url, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(filepath, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"✅ Saved: {filename}")
    except Exception as e:
        print(f"❌ Failed to download {filename}: {e}")

    # polite delay
    time.sleep(random.uniform(1, 3))

print("🎉 Done downloading all files.")


✅ Saved: 10.1126--science.1167748.pdf
✅ Saved: 10.1111--j.1467-9280.1992.tb00678.x.pdf
✅ Saved: 10.1126--science.1121629.pdf
✅ Saved: 10.1037--apl0000183.pdf
✅ Saved: 10.1037--0033-2909.130.5.813.pdf
✅ Saved: 10.1016--j.obhdp.2013.05.001.pdf
✅ Saved: 10.1111--j.1467-9280.2008.02100.x.pdf
✅ Saved: 10.1037--a0035789.pdf
✅ Saved: 10.1016--j.obhdp.2009.01.001.pdf
✅ Saved: 10.1016--j.leaqua.2018.01.002.pdf
✅ Saved: 10.1016--j.beth.2006.03.002.pdf
✅ Saved: 10.1002--job.1992.pdf
✅ Saved: 10.1037--0021-9010.64.6.648.pdf
✅ Saved: 10.1037--h0027407.pdf
✅ Saved: 10.1037--apl0000383.pdf
✅ Saved: 10.1037--pac0000446.pdf
✅ Saved: 10.1037----0021-9010.82.2.247.pdf
✅ Saved: 10.1037--apl0000254.pdf
✅ Saved: 10.1037--bul0000135.pdf
✅ Saved: 10.1037--10165-004.pdf
✅ Saved: 10.1287--mnsc.2021.4219.pdf
✅ Saved: 10.34891--20211105-533.pdf
✅ Saved: 10.1111--ijsa.12344.pdf
✅ Saved: 10.1108--cdi-04-2022-0100.pdf
✅ Saved: 10.1108--jocm-10-2020-0332.pdf
✅ Saved: 10.2307--257060.pdf
✅ Saved: 10.1177--174271501142